In [1]:
import sys
sys.path.append('H:/Tecnologia/EQUIPE - DADOS/dev_env/000_base/database_functions/')
from database import Database
sys.path.append('H:/Tecnologia/EQUIPE - DADOS/dev_env/000_base/modules/')
from utils import Format_data
from datetime import datetime, timedelta
import numpy as np
import pandas as pd


db = Database()
fd = Format_data()

2025-10-16 08:47:53,138 - INFO - Cliente Oracle inicializado com sucesso usando o caminho: H:\Tecnologia\EQUIPE - DADOS\dev_env\000_base\instantclient_19_21
2025-10-16 08:47:53,374 - INFO - Banco de dados conectado com sucesso!


In [10]:
# 🧠 Etapa 1: Consulta única de todas as campanhas -- 
# NOME CAMPANHAS = 
campanhas = ['CAMP_ODONTO_GERAL'] 

for tabela in campanhas:
    print(f"\n🔍 Validando campanha: {tabela}")

    sql = f"""
        SELECT c.*, COUNT(*) OVER () AS TOTAL_REGISTROS
        FROM {tabela} c
        WHERE TO_CHAR(c.LIGAR_EM, 'MM/YYYY') = '10/2025'
        ORDER BY c.LIGAR_EM ASC
    """

    df = db.extract_data(sql)
    print(f"📦 Total de registros extraídos: {df.shape[0]}")




🔍 Validando campanha: CAMP_ODONTO_GERAL


2025-10-16 08:51:13,168 - INFO - Dados coletados com sucesso!


📦 Total de registros extraídos: 469416


In [15]:
# 🧼 Etapa 2: Limpeza e filtragem dos dados

# 1. Limpa o telefone e calcula quantidade de dígitos
df['FONE_LIMPO'] = df['FONE'].str.replace(r'[^0-9]', '', regex=True)
df['QTDE_DIGITOS'] = df['FONE_LIMPO'].str.len()

# 2. Filtra telefones válidos (mais de 10 dígitos e não nulos)
df_fones_validos = df[
    (df['QTDE_DIGITOS'] > 10) &
    (df['FONE_LIMPO'].notnull()) &
    (df['FONE_LIMPO'].str.contains(r'\d', na=False))
].copy()

# 3. Define as colunas que identificam duplicidade
colunas_chave = ['CAMPANHA', 'COD_PACIENTE', 'FONE_LIMPO']

# 4. Marca todos os registros duplicados (inclusive o primeiro)
duplicados = df_fones_validos.duplicated(subset=colunas_chave, keep=False)

# 5. Filtra apenas os registros duplicados
df_duplicados = df_fones_validos[duplicados].copy()

# 6. Exibe a quantidade total de duplicados
print(f"🔁 Total de registros duplicados: {df_duplicados.shape[0]}")

# 7. (Opcional) Ver duplicados agrupados
duplicatas_agrupadas = df_duplicados.groupby(colunas_chave).size().reset_index(name='QTD')
display(duplicatas_agrupadas.sort_values('QTD', ascending=False))

# # 🧹 Etapa 3: Remover duplicados e manter apenas registros únicos
df_sem_duplicados = df_fones_validos[~duplicados].copy()

# # 8. Resultado final
print(f"✅ Registros únicos restantes: {df_sem_duplicados.shape[0]}")


🔁 Total de registros duplicados: 0


,CAMPANHA,COD_PACIENTE,FONE_LIMPO,QTD


✅ Registros únicos restantes: 469377


In [14]:
# Exclui fones duplicados

for tabela in campanhas:
    print(f"\n🔍 Validando campanha: {tabela}")

df_exclui = f""" 
DELETE FROM {tabela}
WHERE ROWID IN (
    SELECT rid FROM (
        SELECT 
            ROWID AS rid,
            ROW_NUMBER() OVER (
                PARTITION BY CAMPANHA, COD_PACIENTE, REGEXP_REPLACE(FONE, '[^0-9]', '')
                ORDER BY LIGAR_EM ASC
            ) AS rn
        FROM CAMP_ODONTO_GERAL
        WHERE TO_CHAR(LIGAR_EM, 'MM/YYYY') = '10/2025'
    )
    WHERE rn > 1
)
 """


🔍 Validando campanha: CAMP_ODONTO_GERAL
